# Week 09: RAG - Retrieval-Augmented Generation
## Give Your AI Agent a Knowledge Base

---

### What is RAG?

**RAG = Retrieve first, then Generate**

```
┌─────────────┐     ┌─────────────────┐     ┌──────────────┐
│   Question  │ --> │ Retrieve Facts  │ --> │   Generate   │
│             │     │ from Knowledge  │     │   Answer     │
└─────────────┘     └─────────────────┘     └──────────────┘
```

**This notebook:**
1. **Part 1:** No RAG - see the problem
2. **Part 2:** Simple RAG - dictionary lookup
3. **Part 3:** Document RAG - search real policy documents

---
## Setup

In [ ]:
!pip install pydantic-ai -q
print("✅ Installed pydantic-ai")

In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")
print("✅ API key set")

In [ ]:
from pydantic import BaseModel
from pydantic_ai import Agent

class ProductListing(BaseModel):
    name: str
    description: str
    price: float

print("✅ Ready")

---
# Part 1: No RAG (The Problem)

What happens when AI doesn't have access to your business knowledge?

In [ ]:
# Agent with NO knowledge access
agent_no_rag = Agent(
    "openai:gpt-4o-mini",
    result_type=ProductListing,
    system_prompt="Create product listings for Utah merchandise."
)

result = await agent_no_rag.run("Create a listing for a red hoodie")

print("═" * 40)
print("NO RAG - AI guesses everything")
print("═" * 40)
print(f"Name:  {result.output.name}")
print(f"Desc:  {result.output.description}")
print(f"Price: ${result.output.price}")

**Problems:**
- ❌ Might use wrong name ("U of U" instead of "University of Utah")
- ❌ Doesn't know our pricing rules (hoodies must be $35+)
- ❌ Missing brand voice (#GoUtes hashtag)

**Solution:** Give the AI access to our knowledge!

---
# Part 2: Simple RAG (Dictionary Lookup)

Store key facts in a dictionary. Agent looks them up before answering.

---
# Part 2: Simple RAG (Dictionary Lookup)

Store key facts in a dictionary. Agent looks them up before answering.

### New Syntax: `@decorator` and Docstrings

**What is `@agent.tool_plain`?**

The `@` symbol is a **decorator** - it modifies the function below it.

```python
@agent_simple_rag.tool_plain      # ← Decorator: "register this function as a tool"
def get_policy(topic: str) -> str:
    """Get policy info."""        # ← Docstring: tells AI what this tool does
    return lookup(topic)
```

| Syntax | What It Does |
|--------|-------------|
| `@agent.tool_plain` | Registers function as a tool the AI can call |
| `"""docstring"""` | Describes the tool to the AI (it reads this!) |
| `topic: str` | Tells AI what input type is expected |
| `-> str` | Tells AI what output type to expect |

**Think of it like:** giving the AI a phone number and instructions for when to call it.

In [ ]:
# Agent WITH simple RAG
agent_simple_rag = Agent(
    "openai:gpt-4o-mini",
    result_type=ProductListing,
    system_prompt="Create U-Shop listings. ALWAYS use get_policy first."
)

@agent_simple_rag.tool_plain
def get_policy(topic: str) -> str:
    """Get policy info. Topics: naming, pricing, tone"""
    return lookup(topic)

result = await agent_simple_rag.run("Create a listing for a red hoodie")

print("═" * 40)
print("SIMPLE RAG - looks up policies")
print("═" * 40)
print(f"Name:  {result.output.name}")
print(f"Desc:  {result.output.description}")
print(f"Price: ${result.output.price}")

**Better!** But what if we have long policy documents, not just short facts?

---
# Part 3: Document RAG (Real Policy Documents)

Real businesses have long documents:
- Brand guidelines (500+ words)
- Compliance policies (legal requirements)

We need to **search** these documents and return only relevant parts.

In [ ]:
# Two policy documents

BRAND_GUIDE = """
UNIVERSITY OF UTAH BRAND GUIDELINES

NAME USAGE:
The official name is "University of Utah". Never use "U of U" or "UofU" 
in merchandise. Always use full name on first reference.

LOGO:
The interlocking U logo must appear on all merchandise. Minimum size 
is 0.5 inches. Never stretch or alter the logo.

COLORS:
Primary: Crimson Red (#CC0000) and White. The crimson must be dominant.
Never substitute with other red shades.

VOICE:
Tone should be spirited, proud, and inclusive. Always end promotional 
copy with #GoUtes. Never disparage other institutions.
"""

COMPLIANCE_POLICY = """
U-SHOP COMPLIANCE REQUIREMENTS

PRICING:
Minimum price for any item is $5.00. T-shirts minimum $20. 
Hoodies minimum $35. Jackets minimum $50. Discounts cannot 
reduce price below these minimums.

PRODUCT STANDARDS:
All products must display official branding. Materials must 
meet quality standards. Size charts required for apparel.

RESTRICTIONS:
No political content. No alcohol references. No disparaging 
content about other schools. Claims like "#1" need citation.
"""

# Store documents in a list
DOCUMENTS = [
    {"name": "Brand Guide", "text": BRAND_GUIDE},
    {"name": "Compliance", "text": COMPLIANCE_POLICY}
]

print(f"📄 Loaded {len(DOCUMENTS)} documents")

In [ ]:
# Search function - finds relevant text in documents
def search_documents(query: str) -> str:
    """Search all documents for relevant information."""
    query = query.lower()
    results = []
    
    for doc in DOCUMENTS:
        # Split document into paragraphs
        paragraphs = doc["text"].strip().split("\n\n")
        
        for para in paragraphs:
            # Check if query words appear in paragraph
            if any(word in para.lower() for word in query.split()):
                results.append(f"[{doc['name']}] {para.strip()}")
    
    if results:
        return "\n\n".join(results[:3])  # Return top 3 matches
    return "No relevant information found."

# Test it
print("Query: 'hoodie price'")
print(search_documents("hoodie price"))

In [ ]:
# Agent WITH document RAG
agent_doc_rag = Agent(
    "openai:gpt-4o-mini",
    result_type=ProductListing,
    system_prompt=(
        "Create U-Shop product listings. "
        "ALWAYS use search_policies to look up brand and compliance "
        "requirements before creating any listing."
    )
)

@agent_doc_rag.tool_plain
def search_policies(query: str) -> str:
    """Search brand guide and compliance docs. Example: 'logo rules' or 'pricing'"""
    return search_documents(query)

result = await agent_doc_rag.run("Create a listing for a crimson hoodie with logo")

print("═" * 40)
print("DOCUMENT RAG - searches policy docs")
print("═" * 40)
print(f"Name:  {result.output.name}")
print(f"Desc:  {result.output.description}")
print(f"Price: ${result.output.price}")

In [ ]:
# Verify it followed the policies
assert result.output.price >= 35.0, "Hoodie must be $35+ per compliance"
assert "#GoUtes" in result.output.description, "Must include #GoUtes per brand guide"
print("✅ Agent followed policies from both documents!")

---
## Summary: Three Levels of RAG

| Level | Knowledge Source | Best For |
|-------|-----------------|----------|
| No RAG | None | ❌ Don't do this |
| Simple RAG | Dictionary | Quick facts, demos |
| Document RAG | Full documents | Real business use |

**The RAG Pattern:**
```
User Question → Search Knowledge → Generate with Context
```

### Practice
1. Add a "returns policy" section to `COMPLIANCE_POLICY`
2. Ask the agent: "What's the return policy?"
3. Verify it retrieves your new section